<a href="https://colab.research.google.com/github/ignacioaranguren1/bd_2/blob/main/bd_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import keras_tuner
import datetime as dt

from tqdm import tqdm
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.layers import BatchNormalization

In [2]:
!sudo fuser /dev/nvidia*
!kill -9 2882

/bin/bash: line 0: kill: (2882) - No such process


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
%matplotlib inline

datapath = r'/content/drive/MyDrive/bd2/data'
os.chdir(datapath)

1.In the data used by Gu, Kelly and Xiu (RFS 2019 – provided in class), use a similar procedure to theirs to predict stock returns with neural networks. Start by finding a suitable baseline configuration, and use a validation procedure to pick optimal hyperparameters for three neural network models: One with 2 hidden layers, one with 3 hidden layers, and one with 4 hidden layers. 

In [5]:
data = pd.read_pickle('returns_chars_panel.pkl')
macro = pd.read_pickle('macro_timeseries.pkl')

In [6]:
def train_validation_test_split(data,train_end_date,validation_end_date):
  tmp = data.reset_index()
  train = tmp[tmp.date<=train_end_date].set_index(['date','permno'],drop=True)
  validation = tmp[(tmp.date>train_end_date) & (tmp.date<=validation_end_date)].set_index(['date','permno'],drop=True)
  test = tmp[tmp.date>validation_end_date].set_index(['date','permno'],drop=True)
  return train,validation,test

In [7]:
data_merged = pd.merge(data,macro,on=['date'])
datelist = list(set(data_merged['date']))
datelist.sort()
data_merged.set_index(['date','permno'],drop=True,inplace=True)

In [8]:
# It is worth mentioning that even though we have set these ratios in order to split the data set the resulting
# weighs are not exactly the same because dates can have more than one observation
train_ratio = 0.5
validation_ratio = 0.25
train_date = datelist[int(len(datelist)*train_ratio)]
validation_date = datelist[int(len(datelist)*(train_ratio+validation_ratio))]
X = data_merged.iloc[:,3:].copy()
y = data_merged['excess_ret'].copy()

In [9]:
X_train,X_validation,X_test = train_validation_test_split(X,train_date,validation_date)
y_train,y_validation,y_test = train_validation_test_split(y,train_date,validation_date)

In [10]:
def keras_model(n_layers, units, learning_rate):
    # Build model
    model = Sequential()
    model.add(layers.Input(shape=(105,)))
    # Add layers iteratively and assign a units hyperparam selector
    for i in range(n_layers):
        model.add(layers.Dense(units=units[0][i], activation='relu'))
        model.add(BatchNormalization()) # Normalizing after activation seems to yield better results
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse')
    return model

class HyperRegressor(keras_tuner.HyperModel):
    def __init__(self, n_layers, *args, **kwargs):
        # Pass all arguments except number of layers to parent
        self.n_layers = n_layers
        super().__init__(*args, **kwargs)

    def build(self, hp):
        # Hyperparameters choices and ranges definition 
        # To increase modularity, we declare units choice for each layer in a list 
        units=[hp.Int(f'units_{i + 1}',min_value=16,max_value=160,step=16) for i in range(self.n_layers)],
        learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
        return keras_model(self.n_layers,
                           units,
                           learning_rate
                          )

    def fit(self, hp, model, x, y, validation_data, **kwargs):
        # Early stop if val_loss does not improve after 3 epochs
        model.fit(x, y, **kwargs)
        x_val, y_val = validation_data
        y_pred = model.predict(x_val)
        # Return a single float to minimize.
        return np.mean((y_pred - y_val)**2)

In [11]:
########################
# CONSTANTS DEFINITION #
########################

MAX_TRIALS = 10
EXECUTION_PER_TRIAL = 3
EPOCHS = 8
BATCH_SIZE = 256

def tune_model(n_layers=2):
  callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)
  tuner = RandomSearch(
        hypermodel=HyperRegressor(n_layers),
        max_trials=MAX_TRIALS,
        executions_per_trial=EXECUTION_PER_TRIAL,
        overwrite=True,
        directory='bd_2',
        project_name=f'NN_new_{n_layers}'
  )
  tuner.search(
      X_train.values, 
      y_train.values,
      validation_data=(X_validation.values, y_validation.values),
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      callbacks=[callback]
  )
  return tuner

In [18]:
models = []
parameters = []
tuners = []
for n in range(2,5):
    tuner = tune_model(n)
    parameters.append(tuner.get_best_hyperparameters)
    models.append(tuner.get_best_models(1)[0])
    tuners.append(tuner)

Trial 10 Complete [00h 09m 55s]
default_objective: 4.852735843562463

Best default_objective So Far: 0.04122103396461683
Total elapsed time: 01h 38m 18s
INFO:tensorflow:Oracle triggered exit


In [13]:
models

In [ ]:
def format_units(buffer_dict):
      # Convert units param to a list of units to match processing formatting
      units = []
      # Check if key is unit, if it is add to list 
      for key, value  in buffer_dict.values.items():
          if 'units' in key:
              units += [value]
      # Crate new dict with correct format 
      best_params = {}
      best_params['units'] = [units]
      best_params['learning_rate'] = buffer_dict['learning_rate']
      return best_params

models_refitted = []
results = {}
for i in range(3):
    # Build and refit model with best params
    callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    best_hps = format_units(tuners[i].get_best_hyperparameters()[0])
    n_layers = len(best_hps['units']) # Get num of hidden layers
    model = keras_model(n_layers, **best_hps) # Rebuild model
    model.fit(X, y, epochs=64, batch_size=256,verbose=True, callbacks=[callback])
    models_refitted.append(model)
    # Evaluate train, val and test 
    train_result = model.evaluate(X_train.values,y_train.values,batch_size=256)
    test_result = model.evaluate(X_test.values,y_test.values,batch_size=256)
    val_result = model.evaluate(X_validation.values,y_validation.values,batch_size=256)
    results[f'NN{i + 1}'] = {'train': train_result, 'validation': val_result, 'test': test_result}

Epoch 1/64
14608/14608 [==============================] - 34s 2ms/step - loss: 0.0320
Epoch 2/64
14608/14608 [==============================] - 33s 2ms/step - loss: 0.0296
Epoch 3/64
14608/14608 [==============================] - 33s 2ms/step - loss: 0.0296
Epoch 4/64
14608/14608 [==============================] - 33s 2ms/step - loss: 0.0296
Epoch 5/64
14608/14608 [==============================] - 33s 2ms/step - loss: 0.0295
Epoch 6/64
14608/14608 [==============================] - 33s 2ms/step - loss: 0.0295
Epoch 7/64
14608/14608 [==============================] - 33s 2ms/step - loss: 0.0295
Epoch 8/64
14608/14608 [==============================] - 33s 2ms/step - loss: 0.0295
Epoch 9/64
14608/14608 [==============================] - 33s 2ms/step - loss: 0.0294
Epoch 10/64
14608/14608 [==============================] - 33s 2ms/step - loss: 0.0294
Epoch 11/64
14608/14608 [==============================] - 33s 2ms/step - loss: 0.0294
Epoch 12/64
14608/14608 [===========================

In [17]:
results

{'NN1': {'test': 0.02526053413748741,
  'train': 0.02225831151008606,
  'validation': 0.03990989550948143},
 'NN2': {'test': 0.025547504425048828,
  'train': 0.022336388006806374,
  'validation': 0.03992683067917824},
 'NN3': {'test': 0.025680555030703545,
  'train': 0.022481001913547516,
  'validation': 0.04027062654495239}}

In [26]:
for i in range(3):
  print(tuners[i].get_best_hyperparameters()[0].values)

{'units_1': 144, 'units_2': 64, 'learning_rate': 0.000256153969803113}
{'units_1': 64, 'units_2': 96, 'units_3': 112, 'learning_rate': 0.0013688978213179357}
{'units_1': 16, 'units_2': 48, 'units_3': 128, 'units_4': 144, 'learning_rate': 0.0005677736913894471}


In [ ]:
with open('models.pkl','wb') as f:
  pickle.dump(models,f)
with open('tuners.pkl','wb') as f:
  pickle.dump(tuners,f)

2.Use test data to get an idea of the out of sample performance of each model. Convert the standard MSE metric for out of sample performance to the “R2 out of sample” metric that was discussed in class. Compare your results to those in Gu-Kelly-Xiu and comment on the differences. 

In [16]:
with open('models.pkl','rb') as f:
  models = pickle.load(f)
with open('tuners.pkl','rb') as f:
  tuners = pickle.load(f)

In [19]:
def r_squared(y_pred, y_test):
    return 1 - np.sum((y_test - y_pred)**2) / np.sum(y_test**2)

rankings = X_test['mvel1'].groupby(['date']).rank()
top_X_test = X_test.loc[rankings<=1000,:].values
top_y_test = y_test.loc[rankings<=1000,:].values
rankings_reverse = X_test['mvel1'].groupby(['date']).rank(ascending=False)
bottom_X_test = X_test.loc[rankings_reverse<=1000,:].values
bottom_y_test = y_test.loc[rankings_reverse<=1000,:].values

R2_oos_df = pd.DataFrame(columns = ['R2_OOS','R2_OOS_top1000','R2_OOS_low1000'],index = ['NN2','NN3','NN4'])
for i in range(3):
    y_pred_all = models_refitted[i].predict(X_test,batch_size=256).reshape(-1,1)
    y_pred_top = models_refitted[i].predict(top_X_test,batch_size=256).reshape(-1,1)
    y_pred_bottom = models_refitted[i].predict(bottom_X_test,batch_size=256).reshape(-1,1)
    
    row = [r_squared(y_pred_all, y_test)[0],
           r_squared(y_pred_top, top_y_test),
           r_squared(y_pred_bottom, bottom_y_test)]
    
    R2_oos_df.iloc[i]= row

In [20]:
R2_oos_df * 100

,R2_OOS,R2_OOS_top1000,R2_OOS_low1000
NN2,1.736162,1.825372,1.188998
NN3,0.619993,1.106661,-0.936966
NN4,0.10239,0.730291,-3.020445


In [ ]:
# X_test.groupby(pd.Grouper(freq='M'))['mvel1'].rank()
best_r_squared = max(R2_oos_df.iloc[:,0])
best_r_squared

0.009491411722372889

3.Pick the model that performs the best out of sample, and interpret its output by doing the following analysis of variable importance:
a.	First, for all stock characteristics, get variable importance by setting one predictor at a time to zero and finding the decrease in out of sample R2. Show a table of the 10 most important variables according to this measure, and give an economic interpretation. 


In [ ]:
feature_importance = {}
for column_name in tqdm(X_train.columns):
    X_tmp = X_test.copy()
    X_tmp[column_name] = 0
    y_pred_temp = models[1].predict(X_tmp,batch_size=256).reshape(-1,1)
    feature_importance[column_name] = best_r_squared - r_squared(y_pred_temp, y_test)[0]
    

  0%|                                                   | 0/105 [00:00<?, ?it/s]

4312/4312 [==============================] - 3s 787us/step


  1%|▍                                          | 1/105 [00:07<12:48,  7.39s/it]

4312/4312 [==============================] - 3s 652us/step


  2%|▊                                          | 2/105 [00:13<11:45,  6.85s/it]

...

 98%|████████████████████████████████████████▏| 103/105 [11:17<00:12,  6.18s/it]

4312/4312 [==============================] - 3s 594us/step


 99%|████████████████████████████████████████▌| 104/105 [11:23<00:06,  6.01s/it]

4312/4312 [==============================] - 3s 610us/step


100%|█████████████████████████████████████████| 105/105 [11:29<00:00,  6.56s/it]


In [ ]:
importance_df = pd.DataFrame.from_dict(feature_importance, orient='index', columns=['magnitude'])
# Retrieve 10 most significant
df_10 = importance_df.sort_values('magnitude').iloc[(len(importance_df['magnitude']) - 10):,]

df_10.index.values

In [ ]:
# Display chart
plt.figure(figsize=(20,5))
plt.barh(np.arange(len(df_10)),df_10.magnitude.values)
plt.yticks(np.arange(len(df_10)),df_10.index.values)
plt.title('Feature Importance',size=16)
plt.show()

b.	Second, get a measure of the joint importance of all our “macro predictors” (i.e., those taken from Welch and Goyal 2008), by setting them all to zero and finding the decrease in out of sample R2. Comment on how important macroeconomic variables are relative to stock characteristics in predicting returns. 

c.	Repeat the two steps above, but by using a measure of the sensitivity of predictions to each input variable, as outlined in the lectures.

4.Fit a penalised linear model (LASSO) to the same data, using validation data to pick the best penalty (e.g., you can use the “sklearn” package in Python to do this easily). Compare its test data performance to the neural network. 

5.Suppose somebody tells you to collect 10 more micro or macro variables that can predict returns and are not in our current dataset. How would you choose those variables, based on the intuitions you have gained in this project?